# OpenIFS 43r3 data

> [!NOTE]
> The online laboratory has only been tested in recent Firefox and Chrome browsers. Some features may not (yet) be supported in Safari browsers.

> [!CAUTION]
> Any changes you make to this notebook will be lost once the page is closed or refreshed. Please download any files you would like to keep.

This notebook is a comprehensive guide to accessing NetCDF files for an OpenIFS simulation. It includes details about the available parameters, levels, and steps, along with instructions for using the `load_OpenIFS_data` function to access these files from the [ECMWF S3 bucket](https://object-store.os-api.cci1.ecmwf.int/esiwacebucket/OpenIFS/). The NetCDF files are not downloaded locally; instead, they are accessed remotely using `kerchunk` and `zarr` via their corresponding `.ref` metadata files (also hosted in the S3 bucket), which were produced up-front using `kerchunk`.

---

## Experiment Details and Available Data Fields

The dataset contains uncompressed IEEE-754 Standard (1985) 64-bit floating-point output from the OpenIFS 43r3 model.
The simulation was conducted on a reduced Gaussian octahedral grid (Tco1279), corresponding to a horizontal resolution of approximately 9 km.
The 5-day forecast was run with a time step length of 600 seconds using a HighResMIP based output configuration. 
The total volume of the available dataset is about 258 GB.


There is a single NetCDF file for the surface level fields and separate files for each pressure level field. This data is published under a Creative Commons Attribution 4.0 International (CC BY 4.0). [https://creativecommons.org/licenses/by/4.0/](https://creativecommons.org/licenses/by/4.0/).

### Pressure Level Parameters

The following parameters are available as pressure level fields:
```python
["z", "t", "q", "w", "vo", "d", "u", "v", "r"]
```

These parameters are stored at the following pressure levels (in Pa):
```python

[100000, 92500, 85000, 70000, 60000, 50000, 40000, 30000, 25000, 20000, 15000, 10000, 7000, 5000, 3000, 2000, 1000, 500, 100]

```
The available time steps start at 2009-10-15 18:00:00 UTC, increasing in 6-hour steps to 2009-10-20 12:00:00 UTC.

### Surface Level Parameters

The following parameters are available as surface level fields:
```python
["10u", "10v", "2d", "2t", "cp", "e", "es", "ewss", "msl", "nsss", "ro", "rsn", "sd", "sf", "skt", "slhf", "smlt", "sp", "sro", "sshf", "ssr", "ssrc", "ssrd", "stl1", "stl2", "stl3", "stl4", "str", "strc", "strd", "swvl1", "swvl2", "swvl3", "swvl4", "tcc", "tciw", "tclw", "tcwv", "tisr", "tp", "tsn", "tsr", "tsrc", "ttr", "ttrc"]
```

Here, the available time steps start at 2009-10-15 15:00:00 UTC, increasing in 3-hour steps to 2009-10-20 12:00:00 UTC.

## Using the `load_OpenIFS_data` Function

To load the NetCDF files, you can use the following function:

```python
load_OpenIFS_data(leveltype=None, param=None)
```

### Parameters

- **`leveltype`**: Defines the type of data to load. Choose one of the following:
  - `"pl"`: Pressure level fields.
  - `"sfc"`: Surface level fields.
- **`param`**: Specifies the parameter of the pressure level data. Options include: `"q"`, `"r"`, `"t"`, `"u"`, `"v"`, `"vo"`, `"w"`, `"v"` and `"z"`.


In [1]:
# URL of S3 bucket
BASE_URL = "https://object-store.os-api.cci1.ecmwf.int/esiwacebucket"

First, the package `zarr` (modern dataset format that is specifically designed for chunked access) and its dependencies need to be imported for the remote access.

In [2]:
import aiohttp
import dask
import fsspec
import zarr

[pyodide]: Memory usage has grown to 71.9MiB (from 49.9MiB) for this notebook


The `load_OpenIFS_data` function simplifies remote access to NetCDF datasets stored on the S3 bucket. It validates input parameters, constructs the correct URL for the requested data, and loads it efficiently as an xarray dataset using the corresponding `.ref` file.


In [3]:
import json

import requests
import xarray as xr


def load_OpenIFS_data(leveltype=None, param=None):
    if leveltype not in {"pl", "sfc"}:
        raise ValueError(f"Invalid leveltype: '{leveltype}'. Available leveltypes: pl, sfc")

    if leveltype == "pl" and not param:
        raise ValueError(f"Param is required for leveltype '{leveltype}'. Available params: q, r, t, u, v, vo, w, z")

    if leveltype == "sfc" and param:
        print("Warning: Specifying 'param' is unnecessary for sfc level and will be ignored.")

    if leveltype == "sfc":
        url = f"{BASE_URL}/OpenIFS/HighResMIP_3h_reduced_sfc.nc"
    elif leveltype == "pl":
        if param not in {"q", "r", "t", "u", "v", "vo", "w", "z"}:
            raise ValueError(f"Incorrect param: '{param}'. Available: q, r, t, u, v, vo, w, z")

        url = f"{BASE_URL}/OpenIFS/HighResMIP_6h_reduced_pl_{param}.nc"
        
    ref = requests.get(f"{url}.ref").json()

    print(f"Loading dataset {url}")

    return xr.open_dataset(
        "reference://", 
        engine="zarr", 
        backend_kwargs={"storage_options": {"fo": ref}},
        consolidated=False, 
        chunks=dict(),
    )

[pyodide]: Memory usage has grown to 149.1MiB (from 71.9MiB) for this notebook


## Example Usage

To load NetCDF file for pressure level parameters, you can use the following example. This will load the **specific humidity (q)** data. 


In [4]:
ds_pl_q = load_OpenIFS_data(leveltype="pl", param="q")
ds_pl_q

Loading dataset https://object-store.os-api.cci1.ecmwf.int/esiwacebucket/OpenIFS/HighResMIP_6h_reduced_pl_q.nc


<xarray.Dataset> Size: 20GB
Dimensions:              (cell: 6599680, nvertex: 4, pressure_levels: 19,
                          time_counter: 20, axis_nbounds: 2)
Coordinates:
    lat                  (cell) float32 26MB dask.array<chunksize=(824960,), meta=np.ndarray>
    lon                  (cell) float32 26MB dask.array<chunksize=(824960,), meta=np.ndarray>
  * pressure_levels      (pressure_levels) float32 76B 1e+05 9.25e+04 ... 100.0
  * time_counter         (time_counter) datetime64[ns] 160B 2009-10-15T18:00:...
    time_instant         (time_counter) datetime64[ns] 160B dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: cell, nvertex, axis_nbounds
Data variables:
    bounds_lat           (cell, nvertex) float32 106MB dask.array<chunksize=(206240, 4), meta=np.ndarray>
    bounds_lon           (cell, nvertex) float32 106MB dask.array<chunksize=(206240, 4), meta=np.ndarray>
    q                    (time_counter, pressure_levels, cell) float64 20GB dask.array<chunksize=(1, 1, 412480), meta=np.ndarray>
    time_counter_bounds  (time_counter, axis_nbounds) datetime64[ns] 320B dask.array<chunksize=(1, 2), meta=np.ndarray>
    time_instant_bounds  (time_counter, axis_nbounds) datetime64[ns] 320B dask.array<chunksize=(1, 2), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    description:  Created by xios
    name:         HighResMIP_6h_reduced_pl_q
    timeStamp:    2024-Apr-05 11:05:21 GMT
    title:        Created by xios
    uuid:         25d5f0a2-ceae-47e1-936f-5b1804228ca6

The `earthkit` package is used for plotting the data.

In [5]:
import earthkit.plots
import earthkit.plots.quickmap

/lib/python3.12/site-packages/earthkit/regrid/utils/caching.py:27: UserWarning: only soft file lock is available
  from filelock import FileLock


[pyodide]: Memory usage has grown to 309.4MiB (from 149.1MiB) for this notebook


The data can be visualized as demonstrated below. For example, the specific humidity at **pressure level 1e5Pa** on **2009-10-20 at 12:00:00 UTC** is plotted. `[::10]` applies slicing to the data being plotted (syntax: `[start:stop:step]`) in order to reduce computational load by selecting only every 10th data point.


In [ ]:
earthkit.plots.quickmap.plot(
    ds_pl_q["q"].rename(dict(time_counter="time")).sel(
        time="2009-10-20T12:00:00", pressure_levels=1e5,
    )[::10], x="lon", y="lat",
);

The surface level data contains all available parameters and can can be loaded as follows.

In [ ]:
ds_sfc = load_OpenIFS_data(leveltype="sfc")
ds_sfc

The data can be visualized as demonstrated below. For example, the 2 metre temperature on **2009-10-20 at 12:00:00 UTC** is plotted.

In [ ]:
earthkit.plots.quickmap.plot(
    ds_sfc["2t"].rename(dict(time_counter="time")).sel(
        time="2009-10-20T12:00:00",
    )[::10], x="lon", y="lat",
);